<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2025/blob/main/DynamicDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [2]:
# 0 Condenser
# 1 Stage 1
# 2 Stage 2
# 3 Feed Stage
# 4 Stage 3
# 5 Stage 4
# 6 Reboiler

N=7
FW_const = 12. #Francis-Weir constant
m0 = 1.2 #moles in each tray at zero flow

In [62]:
from pdb import main
alpha = 2.
F = 1. #mol/s
z = 0.5
q = 0.9
D = 0.6
B = F - D
R = 1.
Lt = R*D  #Stage 1, Stage 2,
mt = (Lt / FW_const)**(2/3) + m0

Lb = Lt + q*F
mb = (Lb / FW_const)**(2/3) + m0

Vt = Lt + D
Vb = Vt - (1-q)*F

#initial conditions
xi = np.full(N, z)
mi = np.r_[10*mt, mt, mt, [mb]*3, 10*mb ]

In [63]:
def rhs(t, vec, R):
    x, m = np.split(vec, 2)
    y = x*alpha/(1-x+x*alpha)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(x)
    V = np.zeros_like(m)
    L = np.zeros_like(m)


    L[0] = R * D
    Vt = R *D + D
    Vb = Vt - (1-q)*F

    V = np.r_[0, [Vt]*3, [Vb]*3]
    L[1:-1] = FW_const*((m[1:-1]-m0)**1.5)

    #proportional controller for bottoms flow from the reboiler
    #adjusts to keep the reboiler holdup to be equal a desired value
    #that is 10 times the holdup of a tray
    Kc = 1.5
    controlled_B = B + Kc*(m[-1]-10*m0)
    L[-1] = controlled_B if controlled_B>0 else 0.


    dm[1:-1] = V[2:] - V[1:-1] + L[:-2] - L[1:-1]
    dm[3]+=F  #3 is the feed stage
    dm[0] = V[1] - D - R*D   #always zero because we have perfect vapor flow control and negligible vapor dynamics
    dm[-1] = L[-2] - L[-1] - V[-1]

    dmx[1:-1] = y[2:]*V[2:] - y[1:-1]*V[1:-1] + x[:-2]*L[:-2] - x[1:-1]*L[1:-1]
    dmx[3]+=z*F  #3 is the feed stage
    dmx[0] = y[1]*V[1] - x[0]*(D + R*D)
    dmx[-1] = x[-2]*L[-2] - x[-1]*L[-1] - y[-1]*V[-1]

    dx = (dmx - x*dm)/m


    return np.r_[dx, dm]



In [64]:
tend = 500.
res=solve_ivp(lambda t, vec: rhs(t, vec, 1), (0,tend), np.r_[xi, mi], method='Radau', dense_output=True)
tplot = np.linspace(0,tend, 50)
res1 = res.sol(tplot)
xs,ms = np.split(res1,2,axis=0)
fig = make_subplots(rows=1, cols=2)
for i, (x, m) in enumerate(zip(xs,ms)):
    fig.add_scatter(x=tplot, y=x, mode='lines', row=1, col=1, name=f'x{i}')
    fig.add_scatter(x=tplot, y=m, mode='lines', row=1, col=2, name=f'm{i}')

fig.update_layout(width=800, height=400, template='plotly_dark')

In [65]:
x_final=xs[:,-1]
y_final=x_final*alpha/(1-x_final+x_final*alpha)
ymc = np.repeat(y_final[1:],2)
xmc = np.r_[x_final[0],np.repeat(x_final[1:-1],2), x_final[-1]]

In [66]:
xplot = np.linspace(0,1, 100)
yplot = xplot*alpha/(1-xplot+xplot*alpha)
fig2 = make_subplots()
fig2.add_scatter(x=xplot,y=yplot, mode='lines')
fig2.add_scatter(x=[0,1], y =[0,1], mode='lines',line_color='rgb(100,100,100)')
fig2.add_scatter(x=xmc, y =ymc, mode='lines',line_color='rgb(0,155,0)')
fig2.update_layout(width=400, height=400, template='plotly_dark', showlegend=False,
                   xaxis=dict(dtick=0.2))

In [67]:
tend = 25.
res=solve_ivp(lambda t, vec: rhs(t, vec, 5), (0,tend), res1[:,-1], method='Radau', dense_output=True)
tplot = np.linspace(0,tend, 50)
res2 = res.sol(tplot)
xs,ms = np.split(res2 ,2,axis=0)
fig = make_subplots(rows=1, cols=2)
for i, (x, m) in enumerate(zip(xs,ms)):
    fig.add_scatter(x=tplot, y=x, mode='lines', row=1, col=1, name=f'x{i}')
    fig.add_scatter(x=tplot, y=m, mode='lines', row=1, col=2, name=f'm{i}')

fig.update_layout(width=800, height=400, template='plotly_dark')

In [68]:
res2.shape

(14, 50)

In [69]:
xs,ms = np.split(res2[:,::16],2,axis=0)

['t=0', 't=16', 't=32', 't=48']

In [75]:
fig3 = make_subplots(rows=1, cols=4, subplot_titles=[f't={t}' for t in np.arange(tplot.size)[::16]])
for i, (x,m) in enumerate(zip(xs.T, ms.T)):
    y=x*alpha/(1-x+x*alpha)
    ymc = np.repeat(y[1:],2)
    xmc = np.r_[x[0],np.repeat(x[1:-1],2), x[-1]]
    fig3.add_scatter(x=xplot,y=yplot, mode='lines', row=1, col=i+1)
    fig3.add_scatter(x=[0,1], y =[0,1], mode='lines',line_color='rgb(100,100,100)', row=1,col=i+1)
    fig3.add_scatter(x=xmc, y =ymc, mode='lines',line_color='rgb(0,155,0)', row=1, col=i+1)
fig3.update_layout(width=700, height=300, template='plotly_dark', showlegend=False)